In [3]:
import os
from PIL import Image

def function_a(text):
    # Replace this function with your custom logic to process the text
    # For example, you could convert the text to uppercase here
    return text.upper()

def process_files(source_directory, target_directory):
    # Get a list of all files in the source directory
    file_list = os.listdir(source_directory)
    
    # Iterate through each file in the list
    for filename in file_list:
        if filename.endswith(".txt"):
            source_file_path = os.path.join(source_directory, filename)
            target_file_path = os.path.join(target_directory, filename)
            
            # Read the content from the source file
            with open(source_file_path, 'r') as txt_file:
                lines = txt_file.readlines()
                xs=[int(line.strip().split(',')[0]) for line in lines]
                ys=[int(line.strip().split(',')[1]) for line in lines]
                image_path = os.path.join(source_directory, filename.replace('.txt','.png'))
                


                b_center_x = (min(xs) + max(xs)) / 2 
                b_center_y = (min(ys) + max(ys)) / 2
                b_width    = (max(xs) - min(xs))
                b_height   = (max(ys) - min(ys))
                
                # Normalise the co-ordinates by the dimensions of the image
                # image_w, image_h, image_c = info_dict["image_size"]
                with Image.open(image_path) as img:
                    image_w, image_h = img.size  
                b_center_x /= image_w 
                b_center_y /= image_h 
                b_width    /= image_w 
                b_height   /= image_h 

                string= "{} {:.3f} {:.3f} {:.3f} {:.3f}".format(0, b_center_x, b_center_y, b_width, b_height)
                    # print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
            # Apply function_a to the content
            # processed_content = function_a(file_content)
            
            # Write the processed content to the target file
            with open(target_file_path, 'w') as target_file:
                target_file.write(string)

# Replace these paths with your desired source and target directories
source_directory_path = 'E:/code/partdp/final'
target_directory_path = 'E:/code/partdp/annotation'

process_files(source_directory_path, target_directory_path)


In [13]:
# Get the annotations
import os
images = ['E:/code/partdp/final/'+x for x in os.listdir('E:/code/partdp/final') if x[-3:] == "png"]
annotations= ['E:/code/partdp/annotation/'+x for x in os.listdir('E:/code/partdp/annotation') if x[-3:] == "txt"]
# images

In [14]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [15]:
images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [9]:
# !mkdir images/train images/val images/test labels/train labels/val labels/test
os.mkdir('images')
os.mkdir('images/train')
os.mkdir('images/val')
os.mkdir('images/test')
os.mkdir('labels')
os.mkdir('labels/train')
os.mkdir('labels/val')
os.mkdir('labels/test')






In [16]:
train_images

['E:/code/partdp/final/1239.png',
 'E:/code/partdp/final/1026.png',
 'E:/code/partdp/final/1177.png',
 'E:/code/partdp/final/2337.png',
 'E:/code/partdp/final/1202.png',
 'E:/code/partdp/final/2021.png',
 'E:/code/partdp/final/2446.png',
 'E:/code/partdp/final/127.png',
 'E:/code/partdp/final/2392.png',
 'E:/code/partdp/final/2741.png',
 'E:/code/partdp/final/697.png',
 'E:/code/partdp/final/1118.png',
 'E:/code/partdp/final/817.png',
 'E:/code/partdp/final/1725.png',
 'E:/code/partdp/final/1609.png',
 'E:/code/partdp/final/2759.png',
 'E:/code/partdp/final/1754.png',
 'E:/code/partdp/final/143.png',
 'E:/code/partdp/final/439.png',
 'E:/code/partdp/final/2534.png',
 'E:/code/partdp/final/772.png',
 'E:/code/partdp/final/588.png',
 'E:/code/partdp/final/1617.png',
 'E:/code/partdp/final/2440.png',
 'E:/code/partdp/final/1614.png',
 'E:/code/partdp/final/229.png',
 'E:/code/partdp/final/1256.png',
 'E:/code/partdp/final/163.png',
 'E:/code/partdp/final/1449.png',
 'E:/code/partdp/final/

In [17]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'labels/train/')
move_files_to_folder(val_annotations, 'labels/val/')
move_files_to_folder(test_annotations, 'labels/test/')
# !mv annotations labels
# %cd ../

In [ ]:
!python train.py --img-size 640 --cfg cfg/training/yolov7.yaml --hyp data/hyp.scratch.custom.yaml --batch 8 --epochs 100 --data data/custom.yaml --weights yolov7_training.pt --workers 24 --name yolo_road_det